In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.subcategory.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                data = {'func': 'navnode_fetch',
                        'payload': '{"jsn":{"tab":"extras","groupname":"' + input_.loc[a, 'Category'] + '","subgroupname":"' + input_.loc[a, 'Subcategory'] + '","nodetype":"subgroupname"}}',
                        'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=data,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(json_data['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))
                
                # = = = = = = = = = = = = = = =
                
                list_type = [type_.strip() for type_ in html.xpath('//td[@class="nlabel"]/a/text()')]
                
                list_type = [re.sub(' +', ' ', type_) for type_ in list_type]
                
                # = = = = = = = = = = = = = = =
                
                list_part = html.xpath('//input[contains(@id, "jsn")]/@value')
                
                # = = = = = = = = = = = = = = =
                
                list_type_code = [json.loads(part)['parttype'].strip() for part in list_part]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Category_No.': int(input_.loc[a, 'Category_No.']),
                                        'Category': input_.loc[a, 'Category'],
                                        'Subcategory_No.': int(input_.loc[a, 'Subcategory_No.']),
                                        'Subcategory': input_.loc[a, 'Subcategory'],
                                        'Type_No.': [i+1 for i in range(len(list_type))],
                                        'Type': list_type,
                                        'Type_Code': list_type_code,
                                        'Tab_Url': ['https://www.rockauto.com/en/tools/' + input_.loc[a, 'Category'] + ',' + input_.loc[a, 'Subcategory'] + ',' + type_ + ',' + type_code for type_, type_code in zip(list_type, list_type_code)]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Category_No.': int(input_.loc[a, 'Category_No.']),
                                     'Category': input_.loc[a, 'Category'],
                                     'Subcategory_No.': int(input_.loc[a, 'Subcategory_No.']),
                                     'Subcategory': input_.loc[a, 'Subcategory']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Category'] + ' - ' + input_.loc[a, 'Subcategory'] + '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Category_No.', 'Subcategory_No.', 'Type_No.'], ascending=[True, True, True]).reset_index(drop=True)
output_correct.to_excel('./3.type-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Category_No.', 'Subcategory_No.'], ascending=[True, True]).reset_index(drop=True)
    output_error.to_excel('./type_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：330

[ok] - Body & Lamp Assembly - Clearance / Side Marker Lamp
[尝试次数：1] - [剩余数量：310] - [当前时间：18:13:47]

[ok] - Belt Drive - Heavy Duty / Industrial V-Belt
[尝试次数：1] - [剩余数量：309] - [当前时间：18:13:47]

[ok] - Belt Drive - Automotive V-Belt
[尝试次数：1] - [剩余数量：308] - [当前时间：18:13:47]

[ok] - Belt Drive - Pulley & Tensioner
[尝试次数：1] - [剩余数量：307] - [当前时间：18:13:47]

[ok] - Body & Lamp Assembly - Body Panel
[尝试次数：1] - [剩余数量：306] - [当前时间：18:13:48]

[ok] - Belt Drive - Light Duty FHP / Lawn & Garden V-Belt
[尝试次数：1] - [剩余数量：305] - [当前时间：18:13:48]

[ok] - Body & Lamp Assembly - Door
[尝试次数：1] - [剩余数量：304] - [当前时间：18:13:48]

[ok] - Body & Lamp Assembly - Back Up / Reverse Lamp
[尝试次数：1] - [剩余数量：303] - [当前时间：18:13:48]

[ok] - Body & Lamp Assembly - Bumper / Pad
[尝试次数：1] - [剩余数量：302] - [当前时间：18:13:48]

[ok] - Belt Drive - Tool
[尝试次数：1] - [剩余数量：301] - [当前时间：18:13:48]

[ok] - Body & Lamp Assembly - Fender
[尝试次数：1] - [剩余数量：300] - [当前时间：18:13:48]

[ok] - Apparel & Gifts - RockAuto Apparel
[尝试次数：1] - [剩余数量：29

[ok] - Electrical - Alternator / Generator
[尝试次数：19] - [剩余数量：207] - [当前时间：18:14:01]

[ok] - Engine - Timing Belt
[尝试次数：2] - [剩余数量：206] - [当前时间：18:14:01]

[ok] - Exhaust & Emission - Clamp
[尝试次数：1] - [剩余数量：205] - [当前时间：18:14:01]

[ok] - Engine - Tool
[尝试次数：1] - [剩余数量：204] - [当前时间：18:14:01]

[ok] - Electrical-Switch & Relay - Relay
[尝试次数：16] - [剩余数量：203] - [当前时间：18:14:01]

[ok] - Engine - Oil Filler
[尝试次数：1] - [剩余数量：202] - [当前时间：18:14:01]

[ok] - Exhaust & Emission - Catalytic Converter
[尝试次数：3] - [剩余数量：201] - [当前时间：18:14:02]

[ok] - Exhaust & Emission - Hanger / Bracket
[尝试次数：2] - [剩余数量：200] - [当前时间：18:14:02]

[ok] - Engine - Gasket
[尝试次数：10] - [剩余数量：199] - [当前时间：18:14:02]

[ok] - Exhaust & Emission - Exhaust System Kit
[尝试次数：4] - [剩余数量：198] - [当前时间：18:14:03]

[ok] - Exhaust & Emission - Muffler
[尝试次数：3] - [剩余数量：197] - [当前时间：18:14:03]

[ok] - Exhaust & Emission - Pipe
[尝试次数：4] - [剩余数量：196] - [当前时间：18:14:03]

[ok] - Exhaust & Emission - Oxygen Sensor
[尝试次数：4] - [剩余数量：195] - [当前时间：18:14:0

[ok] - Hoses/Lines & Clamps - A/C Hose
[尝试次数：1] - [剩余数量：101] - [当前时间：18:14:23]

[ok] - Hardware - O-Ring
[尝试次数：9] - [剩余数量：100] - [当前时间：18:14:23]

[ok] - Heat & Air Conditioning - Fluid / Chemical
[尝试次数：6] - [剩余数量：99] - [当前时间：18:14:23]

[ok] - Hoses/Lines & Clamps - Air Hose
[尝试次数：3] - [剩余数量：98] - [当前时间：18:14:24]

[ok] - Heat & Air Conditioning - Seal
[尝试次数：3] - [剩余数量：97] - [当前时间：18:14:24]

[ok] - Hoses/Lines & Clamps - Copper Tubing
[尝试次数：1] - [剩余数量：96] - [当前时间：18:14:24]

[ok] - Hoses/Lines & Clamps - Air Brake Hose
[尝试次数：5] - [剩余数量：95] - [当前时间：18:14:24]

[ok] - Hoses/Lines & Clamps - Coolant Hose
[尝试次数：3] - [剩余数量：94] - [当前时间：18:14:25]

[ok] - Garage Equipment - Wire
[尝试次数：15] - [剩余数量：93] - [当前时间：18:14:25]

[ok] - Hoses/Lines & Clamps - Air Vent Hose
[尝试次数：1] - [剩余数量：92] - [当前时间：18:14:25]

[ok] - Hoses/Lines & Clamps - Clamp
[尝试次数：4] - [剩余数量：91] - [当前时间：18:14:25]

[ok] - Heat & Air Conditioning - A/C Receiver Drier / Accumulator
[尝试次数：3] - [剩余数量：90] - [当前时间：18:14:25]

[ok] - Heat & Air

[ok] - Wheel - Fluid / Chemical
[尝试次数：8] - [剩余数量：0] - [当前时间：18:14:46]

[ok] - Wiper & Washer - Wiper Blade Refill
[尝试次数：3] - [剩余数量：0] - [当前时间：18:14:46]

[ok] - Transmission-Automatic - O-Ring
[尝试次数：2] - [剩余数量：0] - [当前时间：18:14:46]

[ok] - Wiper & Washer - Fluid / Chemical
[尝试次数：7] - [剩余数量：0] - [当前时间：18:14:47]

[ok] - Wiper & Washer - Wiper Blade
[尝试次数：2] - [剩余数量：0] - [当前时间：18:14:47]

[ok] - Wiper & Washer - Windshield Washer
[尝试次数：7] - [剩余数量：0] - [当前时间：18:14:47]

[ok] - Wheel - Wheel
[尝试次数：11] - [剩余数量：0] - [当前时间：18:14:47]

[ok] - Wiper & Washer - Wiper Motor
[尝试次数：5] - [剩余数量：0] - [当前时间：18:14:48]

[ok] - Wiper & Washer - Wiper Linkage
[尝试次数：7] - [剩余数量：0] - [当前时间：18:14:48]

[ok] - Wheel - Hardware
[尝试次数：13] - [剩余数量：0] - [当前时间：18:14:48]

[ok] - Wheel - Tool
[尝试次数：2] - [剩余数量：0] - [当前时间：18:14:49]

[ok] - Hardware - Washer
[尝试次数：14] - [剩余数量：0] - [当前时间：18:14:59]

[ok] - Interior - Cargo Management
[尝试次数：27] - [剩余数量：0] - [当前时间：18:15:15]

输出ing...

Done ~
